In [1]:
import reservoirflow as rf
import numpy as np

In [2]:
type(rf.scalers.MinMax((1, 1)))

reservoirflow.scalers.minmax.MinMax

In [3]:
def create_model_example_7_1():
    grid = rf.grids.Cartesian(
        nx=4, ny=1, nz=1, dx=300, dy=350, dz=40, phi=0.27, kx=270, dtype="double"
    )
    fluid = rf.fluids.SinglePhase(mu=0.5, B=1, dtype="double")
    model = rf.models.Numerical(grid, fluid, dtype="double", verbose=False)
    model.set_well(id=4, q=-600, s=1.5, r=3.5)
    model.set_boundaries({0: ("pressure", 4000), 5: ("rate", 0)})
    return model


def create_model():
    grid = rf.grids.Cartesian(
        nx=3,
        ny=1,
        nz=1,
        dx=300,
        dy=350,
        dz=20,
        phi=0.27,
        kx=1,
        ky=1,
        kz=0.1,
        comp=1 * 10**-6,
        dtype="double",
    )
    fluid = rf.fluids.SinglePhase(
        mu=0.5, B=1, rho=50, comp=1 * 10**-5, dtype="double"
    )
    model = rf.models.Numerical(
        grid, fluid, pi=6000, dt=5, start_date="10.10.2018", dtype="double"
    )

    n_samples = 1
    cells_id = model.grid.get_cells_id(False, True)[-1].flatten()
    wells = np.random.choice(cells_id, n_samples, False)
    for id in wells:
        model.set_well(id=id, q=-300, pwf=100, s=1.5, r=3.5)

    wells = np.random.choice(cells_id, n_samples, False)
    for id in wells:
        model.set_well(id=id, q=100, s=0, r=3.5)

    return model


model = create_model()
model.run(10, isolver="cgs")

[info] Simulation run started: 10 timesteps.


[step]: 10 - : 100%|██████████| 10/10 [00:00<00:00, 61.21steps/s]

[info] Simulation run of 10 steps finished in 0.17 seconds.
[info] Material Balance Error: 6.8833827526759706e-15.


In [4]:
arr = model.grid.get_cells_center(True, False)
print(arr.shape)
print(arr.min(axis=0))
arr

(5, 3)
[150. 175.  10.]


array([[ 150.,  175.,   10.],
       [ 450.,  175.,   10.],
       [ 750.,  175.,   10.],
       [1050.,  175.,   10.],
       [1350.,  175.,   10.]])

In [5]:
np.count_nonzero(np.array([1, 2, 0, 0]) == 0)

2

In [6]:
scaler = rf.scalers.MinMax((-1, 1)).fit(arr, axis=0)
scaler.transform(arr)

c:\users\abugrzka\projects\reservoirflow\reservoirflow\scalers\minmax.py:38: RuntimeWarning: invalid value encountered in true_divide
  self.vmax - self.vmin


array([[-1. ,  nan,  nan],
       [-0.5,  nan,  nan],
       [ 0. ,  nan,  nan],
       [ 0.5,  nan,  nan],
       [ 1. ,  nan,  nan]])

In [7]:
t = np.arange(0, (model.tstep + 1) * model.dt, model.dt)
rf.scalers.MinMax((0, 1)).fit_transform(t, axis=0)

array([0. , 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1. ])

In [8]:
t

array([ 0,  5, 10, 15, 20, 25, 30, 35, 40, 45, 50])

In [9]:
t, x = model.get_domain(False, True)
t, x

(array([[ 0],
        [ 5],
        [10],
        [15],
        [20],
        [25],
        [30],
        [35],
        [40],
        [45],
        [50]]),
 array([[ 150.],
        [ 450.],
        [ 750.],
        [1050.],
        [1350.]]))

In [10]:
rf.scalers.MinMax((-1, 1)).fit_transform(x, axis=0)

array([[-1. ],
       [-0.5],
       [ 0. ],
       [ 0.5],
       [ 1. ]])

In [11]:
rf.scalers.MinMax((0, 1)).fit_transform(t, axis=0)

array([[0. ],
       [0.1],
       [0.2],
       [0.3],
       [0.4],
       [0.5],
       [0.6],
       [0.7],
       [0.8],
       [0.9],
       [1. ]])

In [12]:
model.get_centers(False)

(array([[ 150.],
        [ 450.],
        [ 750.],
        [1050.],
        [1350.]]),
 ['x'])

In [13]:
config = {"time": ["MinMax", (0, 1)], "space": ["minMax"]}

In [14]:
model.set_scalers({"time": ["minmax", (-10, 10)], "rate": ["minmax", None]})
model.scalers_dict

{'time': ['MinMax', (-10, 10)],
 'space': ['MinMax', (-1, 1)],
 'pressure': ['MinMax', (-1, 1)],
 'rate': [None, None]}

In [15]:
model.update_scalers(False)
model.pressures_scaler.transform(model.pressures[1])
# model.space_scaler.transform(model.get_centers(False)[0])

array([0.82200363, 0.84839774, 0.95989506, 0.298079  , 0.82200363])

In [16]:
model.get_df(
    scale=True,
    melt=False,
    boundary=True,
    units=True,
    columns=["time", "cells", "wells"],
)

,Time [Scaled],Q2 [Scaled],Q3 [Scaled],P0 [Scaled],P1 [Scaled],P2 [Scaled],P3 [Scaled],P4 [Scaled],Qw3 [Scaled],Qw2 [Scaled],Pwf3 [Scaled],Pwf2 [Scaled]
Step,,,,,,,,,,,,
0,-10.0,0.0,0.000000,0.822004,0.822004,0.822004,0.822004,0.822004,0.000000,0.0,0.822004,0.822004
1,-8.0,100.0,-212.053390,0.822004,0.848398,0.959895,0.298079,0.822004,-212.053390,100.0,-3.385090,2.319179
2,-6.0,100.0,-191.218025,0.822004,0.877416,1.000000,-0.063812,0.822004,-191.218025,100.0,-3.385090,2.359284
3,-4.0,100.0,-176.459962,0.822004,0.898867,0.989482,-0.320145,0.822004,-176.459962,100.0,-3.385090,2.348766
4,-2.0,100.0,-165.739007,0.822004,0.909444,0.954126,-0.506358,0.822004,-165.739007,100.0,-3.385090,2.313410
5,0.0,100.0,-157.752835,0.822004,0.909129,0.907795,-0.645071,0.822004,-157.752835,100.0,-3.385090,2.267079
6,2.0,100.0,-151.654964,0.822004,0.899295,0.857754,-0.750985,0.822004,-151.654964,100.0,-3.385090,2.217038
7,4.0,100.0,-146.884997,0.822004,0.881756,0.807664,-0.833835,0.822004,-146.884997,100.0,-3.385090,2.166948
8,6.0,100.0,-143.065174,0.822004,0.858305,0.759243,-0.900182,0.822004,-143.065174,100.0,-3.385090,2.118527
